#Import

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

#Get Data

In [ ]:
# Path to the Excel file in Google Drive
#excel_file_path = "/content/drive/MyDrive/swuds/t5_thesis/98_progress/dataset/nd_sales.xlsx"

# Read the Excel file into a pandas DataFrame
#df = pd.read_excel(excel_file_path)

In [ ]:
# Read the CSV file into a pandas DataFrame
df = pd.read_excel("/content/nd_sales.xlsx")

In [ ]:
# prompt: prompt: ดูว่า df มี NaN กี่ row

# ดูจำนวน row ที่มี NaN ในแต่ละคอลัมน์
print(df.isnull().sum())

# ดูจำนวน row ทั้งหมดใน DataFrame ที่มีอย่างน้อยหนึ่งค่า NaN
print("Total rows with NaN:", df.isnull().any(axis=1).sum())

In [ ]:
# Drop rows with missing values
df_cleaned = df.dropna()
df = df_cleaned.copy()

In [ ]:
df.info()

In [ ]:
# Convert date columns
df['new_datetime'] = pd.to_datetime(df['date'])

In [ ]:
# Drop unwanted columns
df = df.drop(columns=['local_event', 'other_event']) #'local_event', 'other_event' เป็นคอลัมน์ที่ถูกเพิ่มมาทีหลังได้ไม่นาน อาจจะทำให้ประสิทธิภาพการวิจัยลดลง

In [ ]:
# Remove commas from numeric strings
for col in df.columns:
   if df[col].dtype == 'object':
         df[col] = df[col].astype(str).str.replace(',', '', regex=False)

In [ ]:
# Convert columns to appropriate data types
df['day'] = df['day'].astype('object')
df['date'] = pd.to_datetime(df['date'])
df['cost_change'] = pd.to_numeric(df['cost_change'], errors='coerce').astype('float64')
df['cost_material'] = pd.to_numeric(df['cost_material'], errors='coerce').astype('float64')
df['cost_banking'] = pd.to_numeric(df['cost_banking'], errors='coerce').astype('float64')
df['cost_sum'] = pd.to_numeric(df['cost_sum'], errors='coerce').astype('float64')
df['sales_fs'] = pd.to_numeric(df['sales_fs'], errors='coerce').astype('float64')
df['sales_qr'] = pd.to_numeric(df['sales_qr'], errors='coerce').astype('float64')
df['sales_gb'] = pd.to_numeric(df['sales_gb'], errors='coerce').astype('float64')
df['sales_sp'] = pd.to_numeric(df['sales_sp'], errors='coerce').astype('float64')
df['sales_lm'] = pd.to_numeric(df['sales_lm'], errors='coerce').astype('float64')
df['sales_rb'] = pd.to_numeric(df['sales_rb'], errors='coerce').astype('float64')
df['sales_sum'] = pd.to_numeric(df['sales_sum'], errors='coerce').astype('float64')
df['profit'] = pd.to_numeric(df['profit'], errors='coerce').astype('float64')
df['promotion_01'] = df['promotion_01'].astype('object')
df['promotion_02'] = df['promotion_02'].astype('object')
df['ismarketday'] = pd.to_numeric(df['ismarketday'], errors='coerce').fillna(0).astype('int64')
df['isschoolday'] = pd.to_numeric(df['isschoolday'], errors='coerce').fillna(0).astype('int64')
df['holiday'] = pd.to_numeric(df['holiday'], errors='coerce').fillna(0).astype('int64')
df['isbuddaday'] = pd.to_numeric(df['isbuddaday'], errors='coerce').fillna(0).astype('int64')
df['nd_lottery'] = pd.to_numeric(df['nd_lottery'], errors='coerce').fillna(0).astype('int64')
df['applylkcolor'] = df['applylkcolor'].astype('object')
df['PP'] = pd.to_numeric(df['PP'], errors='coerce').astype('float64')
df['T'] = pd.to_numeric(df['T'], errors='coerce').astype('float64')
df['H'] = pd.to_numeric(df['H'], errors='coerce').astype('float64')
df['V'] = pd.to_numeric(df['V'], errors='coerce').astype('float64')
df['pm2.5'] = pd.to_numeric(df['pm2.5'], errors='coerce').astype('float64')
df['pm10'] = pd.to_numeric(df['pm10'], errors='coerce').astype('float64')
df['o3'] = pd.to_numeric(df['o3'], errors='coerce').astype('float64')
df['no2'] = pd.to_numeric(df['no2'], errors='coerce').astype('float64')

#EDA

In [ ]:
df.info()

ตอนนี้ทุก columns ชนิดถูกแล้ว ลองดูข้อมูลต่อ

In [ ]:
df.head(10)

In [ ]:
df.describe()

In [ ]:
# Merge banking cost into material cost
df['cost_material'] = df['cost_material'] + df['cost_banking']
df = df.drop(columns=['cost_banking'])

In [ ]:
# Merge QR sales into FS sales
df['sales_qr'] = df['sales_qr'].fillna(0)
df['sales_fs'] = df['sales_fs'].fillna(0)
df['sales_fs'] = df['sales_qr'] + df['sales_fs']
df = df.drop(columns=['sales_qr'])

In [ ]:
df.describe()

In [ ]:
# Map Thai day abbreviations to English
day_mapping = {
            'อา.': 'Sun',
            'จ.': 'Mon',
            'อ.': 'Tue',
            'พ.': 'Wed',
            'พฤ.': 'Thu',
            'ศ.': 'Fri',
            'ส.': 'Sat'
        }
df['day'] = df['day'].replace(day_mapping)

In [ ]:
df.head(5)

ลองดูข้อมูลในมุมมองกราฟบ้าง

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df['date'], df['sales_sum'])
plt.xlabel('date')
plt.ylabel('sales_sum')
plt.title('Sales Sum over Time')
plt.grid(True)
plt.show()

In [ ]:
df = df[df['sales_sum'] != 0]

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df['date'], df['sales_sum'])
plt.xlabel('date')
plt.ylabel('sales_sum')
plt.title('Sales Sum over Time')
plt.grid(True)
plt.show()

In [ ]:
# Calculate the average sales_sum for each day of the week
average_sales_by_day = df.groupby('day')['sales_sum'].mean().reindex(['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat'])

# Plot the linear graph
plt.figure(figsize=(10, 6))
plt.plot(average_sales_by_day.index, average_sales_by_day.values, marker='o', linestyle='-')
plt.xlabel('Day of the Week')
plt.ylabel('Average Sales Sum')
plt.title('Average Sales Sum by Day of the Week')
plt.grid(True)
plt.show()

In [ ]:
# Adjust Friday sales by adding 6000
df.loc[df['day'] == 'Fri', 'sales_sum'] = df.loc[df['day'] == 'Fri', 'sales_sum'] + 5000

In [ ]:
# Calculate the average sales_sum for each day of the week
average_sales_by_day = df.groupby('day')['sales_sum'].mean().reindex(['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat'])

# Plot the linear graph
plt.figure(figsize=(10, 6))
plt.plot(average_sales_by_day.index, average_sales_by_day.values, marker='o', linestyle='-')
plt.xlabel('Day of the Week')
plt.ylabel('Average Sales Sum')
plt.title('Average Sales Sum by Day of the Week')
plt.grid(True)
plt.show()

ข้อมูล sales_sum ถูกต้องแล้ว ลองดู outliner ต่อ

In [ ]:
# Select only the numeric columns
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
# Create a single figure and axes for the box plots
plt.figure(figsize=(15, 10))
# Plot box plots for each numeric column
df[numeric_cols].boxplot(vert=False, patch_artist=True, showfliers=True) # vert=False for horizontal box plots
plt.title('Box Plot of Numeric Variables')
plt.xlabel('Value')
plt.ylabel('Numeric Variables')
plt.tight_layout() # Adjust layout to prevent labels overlapping
plt.show()

ไม่มี out liner ที่ผิดปกติเลย

In [ ]:
# prompt: สร้าง Seasonal Plot (เช่น ยอดขายเฉลี่ยรายวันของสัปดาห์) เพื่อยืนยันรูปแบบรายสัปดาห์ แยกเป็น 6 กราฟ แสดงเฉพาะ Sun - Fri อย่างละกราฟ

# Extract day of the week and year
df['dayofweek'] = df['new_datetime'].dt.dayofweek  # Monday=0, Sunday=6
df['year'] = df['new_datetime'].dt.year

# Filter out Saturday (dayofweek == 5)
df_filtered_days = df[df['dayofweek'] != 5].copy()

# Map dayofweek to English day names
day_map = {
    0: 'Mon',
    1: 'Tue',
    2: 'Wed',
    3: 'Thu',
    4: 'Fri',
    5: 'Sat',
    6: 'Sun'
}
df_filtered_days['day_name'] = df_filtered_days['dayofweek'].map(day_map)

# Get the list of days to plot (Sun to Fri)
days_to_plot = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri']

# Create subplots for each day
fig, axes = plt.subplots(nrows=len(days_to_plot), ncols=1, figsize=(12, 3 * len(days_to_plot)))
fig.suptitle('Seasonal Plot of Average Daily Sales Sum (Sun - Fri)', y=1.02)

for i, day in enumerate(days_to_plot):
    # Filter data for the current day
    df_day = df_filtered_days[df_filtered_days['day_name'] == day].copy()

    # Group by date and calculate the average sales sum for that date
    # (even though there's usually only one entry per date, this handles potential duplicates gracefully)
    daily_avg_sales = df_day.groupby('new_datetime')['sales_sum'].mean()

    # Plot the average daily sales for the current day
    axes[i].plot(daily_avg_sales.index, daily_avg_sales.values, label=day)
    axes[i].set_title(f'Average Sales Sum on {day}')
    axes[i].set_ylabel('Average Sales Sum')
    axes[i].grid(True)
    axes[i].legend()

# Set a common xlabel for all subplots
fig.text(0.5, 0.00, 'Date', ha='center')

plt.tight_layout()
plt.show()

In [ ]:
# Set the 'date' column as the index
df_decomp = df.set_index('date')

# Select the time series data for decomposition
time_series = df_decomp['sales_sum']

# Perform additive decomposition
decomposition = seasonal_decompose(time_series, model='additive', period=7) # Use period=7 for weekly seasonality

# Plot the decomposed components
fig = decomposition.plot()
fig.set_size_inches(10, 8)
plt.tight_layout()
plt.show()


In [ ]:
# prompt: สร้าง Decomposition Plot เพื่อแยกองค์ประกอบของ Trend, Seasonality และ Residuals ออกจากกันอย่างชัดเจน โดยทำเป็นรายเดือน

# Perform additive decomposition with monthly period
decomposition_monthly = seasonal_decompose(time_series, model='additive', period=30) # Use period=30 for roughly monthly seasonality

# Plot the decomposed components for monthly
fig_monthly = decomposition_monthly.plot()
fig_monthly.set_size_inches(10, 8)
fig_monthly.suptitle('Monthly Decomposition of Sales Sum', y=1.02) # Add a specific title
plt.tight_layout()
plt.show()

In [ ]:
# เพื่อแยกตามเดือนตามปฏิทินจริงๆ เราสามารถ Resample ข้อมูลเป็นรายเดือนก่อน แล้วจึงทำ Decomposition
# Resample to monthly frequency, summing the sales_sum for each month
monthly_data = time_series.resample('M').sum()

# A better approach for visualizing calendar month patterns is a seasonal plot by month.

# We already have a 'new_datetime' column. Let's use that to extract the month.
df['month'] = df['new_datetime'].dt.month

# Group by month and calculate the average sales sum for each month across all years
average_sales_by_month = df.groupby('month')['sales_sum'].mean()

# Map month numbers to names for better plotting
month_names = {
    1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
    7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
}
average_sales_by_month.index = average_sales_by_month.index.map(month_names)

# Plot the average sales by calendar month
plt.figure(figsize=(10, 6))
plt.plot(average_sales_by_month.index, average_sales_by_month.values, marker='o', linestyle='-')
plt.xlabel('Calendar Month')
plt.ylabel('Average Sales Sum')
plt.title('Average Sales Sum by Calendar Month Across All Years')
plt.grid(True)
plt.show()

# Let's do that as well, as it aligns more with the concept of decomposition plots.
# Resample to monthly frequency (sum of sales for each month)
monthly_series = time_series.resample('M').sum()

# Perform additive decomposition on the monthly aggregated data with period=12
# This will show trend and annual seasonality (12-month cycle) within the monthly sums
decomposition_monthly_agg = seasonal_decompose(monthly_series, model='additive', period=12)

# Plot the decomposed components for monthly aggregated data
fig_monthly_agg = decomposition_monthly_agg.plot()
fig_monthly_agg.set_size_inches(10, 8)
fig_monthly_agg.suptitle('Decomposition of Monthly Sales Sum (Annual Seasonality)', y=1.02) # Add a specific title
plt.tight_layout()
plt.show()

In [ ]:
# prompt: plot line graph แบ่งตามเดือน แกน x = Jan, Feb, ... , แกน y = sales_sum โดยมีหลายเส้นแบ่งตามปีของข้อมูล เช่น 2023, 2024, 2025 (ไม่ต้องพลอตเดือนสุดท้ายของปีสุดท้าย)

# Group by year and month, then sum sales_sum
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

# Create monthly data grouped by year
monthly_sales_by_year = df.groupby(['year', 'month'])['sales_sum'].sum().reset_index()

# Get the maximum year in the dataset
max_year = monthly_sales_by_year['year'].max()

# Filter out the last month of the last year
monthly_sales_filtered = monthly_sales_by_year[
    ~((monthly_sales_by_year['year'] == max_year) & (monthly_sales_by_year['month'] == 12))
].copy()


plt.figure(figsize=(12, 7))

# Iterate through each year and plot a line
for year in monthly_sales_filtered['year'].unique():
    df_year = monthly_sales_filtered[monthly_sales_filtered['year'] == year]

    # Ensure all months (1-12) are present for consistent plotting
    # Create a full set of months for the year
    full_months = pd.DataFrame({'month': range(1, 13)})
    df_year_full = pd.merge(full_months, df_year, on='month', how='left')

    # Sort by month
    df_year_full = df_year_full.sort_values('month')

    # Plot the data, excluding the last month if it's the max year and month 12
    months_to_plot = range(1, 13)
    if year == max_year:
        months_to_plot = range(1, 12) # Exclude month 12 for the last year

    plt.plot(df_year_full['month'].iloc[months_to_plot.start-1 : months_to_plot.stop-1], # Adjust indexing for months
             df_year_full['sales_sum'].iloc[months_to_plot.start-1 : months_to_plot.stop-1],
             marker='o', linestyle='-', label=str(year))


# Set x-axis labels to month names
month_names_short = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
plt.xticks(range(1, 13), month_names_short) # Set ticks for all 12 months

plt.xlabel('Month')
plt.ylabel('Sum of Sales')
plt.ylim(ymin=0)
plt.title('Monthly Sales Sum by Year')
plt.legend(title='Year')
plt.grid(True)
plt.show()


In [ ]:
# prompt: คำนวณ Correlation Matrix จาก numeric features

import seaborn as sns

# Select only the numeric columns again for correlation calculation
numeric_cols_for_corr = df.select_dtypes(include=['float64', 'int64']).columns

# Calculate the correlation matrix
correlation_matrix = df[numeric_cols_for_corr].corr()

# Print the correlation matrix
print("Correlation Matrix of Numeric Features:")
print(correlation_matrix)

# Optional: Visualize the correlation matrix using a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".1f", linewidths=.5)
plt.title('Correlation Heatmap of Numeric Features')
plt.show()

In [ ]:
#df.to_csv('nd_sales_clean.csv', index=False)
#print("DataFrame exported successfully as 'nd_sales_clean.csv'")

In [ ]:
# prompt: แสดง VIF (Variance Inflation Factor)

from statsmodels.stats.outliers_influence import variance_inflation_factor

# Select numerical features for VIF calculation
# Exclude the target variable ('sales_sum') and potentially highly correlated/dependent variables like profit, costs, etc.
# Also exclude variables that are essentially identifiers or not meant to be predictors like date, year, month, dayofweek, day_name
# Let's select relevant numerical predictors identified from EDA/correlation analysis.
# Based on the previous analysis, potential numerical predictors could include:
# ismarketday, isschoolday, holiday, isbuddaday, nd_lottery,
# sales from other channels (sales_gb, sales_rb) - but remember to use lagged values for prediction!
# Weather/pollution variables (T, H, V, PP, pm2.5, o3, no2) - consider multicollinearity

# For VIF calculation, we need the predictor variables.
# Let's start with a subset of potential numerical predictors excluding the target and cost/profit.
# Also, let's be mindful of multicollinearity issues identified (e.g., pm2.5 and pm10).
# We'll use a subset of potentially useful features for this example.
# Note: VIF is calculated for the independent variables *before* modeling.

# First, identify numerical columns that are potential predictors
# Exclude 'sales_sum', 'profit', and cost-related columns as they are results/components of sales.
# Exclude date-related columns that are not numerical predictors themselves but used for time series analysis.
# Exclude 'dayofweek', 'year', 'month' as we might use categorical encoding for time-based features or rely on the time series model itself.
numerical_predictors = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Remove the target variable and other dependent/related variables
exclude_cols = ['sales_sum', 'profit', 'cost_sum', 'cost_material', 'cost_change',
                'date', 'new_datetime', 'year', 'month', 'dayofweek']

vif_features = [col for col in numerical_predictors if col not in exclude_cols]

# Optional: Further refine vif_features based on EDA/correlation.
# For instance, if pm10 is highly correlated with pm2.5 and we decide to use only pm2.5.
# Let's remove pm10 for this VIF calculation based on previous observation (corr=0.86 with pm2.5).
if 'pm10' in vif_features:
    vif_features.remove('pm10')

# Create a DataFrame with only the features for VIF calculation
X = df[vif_features]

# Ensure there are no missing values in the selected columns for VIF calculation
X = X.dropna()

# Handle potential infinite values that might arise from perfectly correlated variables
# A common issue is when one variable is a linear combination of others.
# Check for constant columns or columns that are linear combinations.
# A simple check is to see if the variance is zero or close to zero.
non_constant_cols = X.columns[X.var() > 1e-6] # Remove columns with near-zero variance
X = X[non_constant_cols]

# If after removing near-zero variance columns, there are still perfect linear combinations,
# VIF calculation will fail. This might require domain knowledge or further analysis
# to identify and remove one of the perfectly correlated variables.
# For example, if 'A', 'B', 'C' are such that A + B = C, then including all three will cause issues.

# Calculate VIF for each predictor
# Make sure X is not empty
if not X.empty:
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    # calculating VIF for each feature
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                       for i in range(len(X.columns))]

    print("\nVariance Inflation Factor (VIF):")
    print(vif_data)
else:
    print("No suitable numerical features found for VIF calculation after exclusions.")



In [ ]:
df_holiday = df[(df['day'] == 'Sun') | (df['holiday'] == 1)].copy()

In [ ]:
# Filter data for 2024 and 2025
df_2024 = df[df['year'] == 2023].copy()
df_2025 = df[df['year'] == 2024].copy()

# Calculate average daily sales per month for 2023
average_sales_2024 = df_2024.groupby(df_2024['date'].dt.month)['sales_sum'].mean()

# Calculate average daily sales per month for 2025
average_sales_2025 = df_2025.groupby(df_2025['date'].dt.month)['sales_sum'].mean()

# Combine the average sales into a single DataFrame
comparison_df = pd.DataFrame({
    'Average Sales 2024': average_sales_2024,
    'Average Sales 2025': average_sales_2025
})

# Calculate the percentage change
comparison_df['Percentage Change (%)'] = ((comparison_df['Average Sales 2025'] - comparison_df['Average Sales 2024']) / comparison_df['Average Sales 2024']) * 100

# Map month numbers to names
month_names = {
    1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
    7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
}
comparison_df.index = comparison_df.index.map(month_names)

# Display the comparison
print("Average Daily Sales Comparison (2025 vs 2024):")
display(comparison_df)

In [ ]:
# Filter data for 2024 and 2025
df_2024 = df_holiday[df_holiday['year'] == 2024].copy()
df_2025 = df_holiday[df_holiday['year'] == 2025].copy()

# Calculate average daily sales per month for 2023
average_sales_2024 = df_2024.groupby(df_2024['date'].dt.month)['sales_sum'].mean()

# Calculate average daily sales per month for 2025
average_sales_2025 = df_2025.groupby(df_2025['date'].dt.month)['sales_sum'].mean()

# Combine the average sales into a single DataFrame
comparison_df = pd.DataFrame({
    'Average Sales 2024': average_sales_2024,
    'Average Sales 2025': average_sales_2025
})

# Calculate the percentage change
comparison_df['Percentage Change (%)'] = ((comparison_df['Average Sales 2025'] - comparison_df['Average Sales 2024']) / comparison_df['Average Sales 2024']) * 100

# Map month numbers to names
month_names = {
    1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
    7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
}
comparison_df.index = comparison_df.index.map(month_names)

# Display the comparison
print("Average Daily Sales Comparison (2025 vs 2024):")
display(comparison_df)

In [ ]:
# Filter data for Sundays or holidays in 2024
df_holiday_2024 = df_holiday[df_holiday['year'] == 2024].copy()

# Filter data for Sundays or holidays in 2025
df_holiday_2025 = df_holiday[df_holiday['year'] == 2025].copy()

# Calculate average daily sales for Sundays or holidays in 2024
average_holiday_sales_2024 = df_holiday_2024['sales_sum'].mean()

# Calculate average daily sales for Sundays or holidays in 2025
average_holiday_sales_2025 = df_holiday_2025['sales_sum'].mean()

# Calculate the percentage change
if average_holiday_sales_2024 != 0:
    percentage_change = ((average_holiday_sales_2025 - average_holiday_sales_2024) / average_holiday_sales_2024) * 100
else:
    percentage_change = float('inf') # Handle division by zero if 2024 average is 0

print(f"Average daily sales on Sundays or holidays in 2024: {average_holiday_sales_2024:.2f}")
print(f"Average daily sales on Sundays or holidays in 2025: {average_holiday_sales_2025:.2f}")
print(f"Percentage change in average daily sales on Sundays or holidays (2025 vs 2024): {percentage_change:.2f}%")